In [1]:
!pip install -U sentence-transformers
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.4 MB 10.9 MB/s 
     |████████████████████████████████| 1.2 MB 57.9 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 596 kB 72.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e070416d977c8b521ec72ad5d331001417f757cab3182f97617d4a1b044d69d1
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
temp_df = pd.read_csv("/content/drive/MyDrive/clean_GLA.csv")
temp_df["words"] = temp_df["words"].apply(lambda x: x[2:-2].split("', '"))

In [41]:
temp_df

,words,title,dep
0,"[cours, focus, identifi, financi, object, firm...",Finance 1 ACCFIN1003,Accounting and Finance
1,"[gener, cours, provid, challeng, interest, int...",Financial Accounting 1 ACCFIN1004,Accounting and Finance
2,"[reassess, avail, group, assign, gener, cours,...",Introduction to Business Reporting & Financial...,Accounting and Finance
3,"[cours, aim, provid, introduct, basic, concept...","Introduction to Finance, Investments and Insti...",Accounting and Finance
4,"[cours, encourag, acquisit, knowledg, relat, a...",Introduction to Management Accounting and Cont...,Accounting and Finance
...,...,...,...
5217,"[cours, aim, upskil, practis, veterinarian, eq...",Veterinary Practitioners and the Food Supply C...,Veterinary Med and Surgery
5218,"[provid, particip, knowledg, understand, exist...",Zoonoses And Infectious Disease VETMED5023,Veterinary Med and Surgery
5219,"[overal, cours, provid, student, broad, knowle...",Zoonotic Diseases VETMED5053,Veterinary Med and Surgery
5220,"[profession, phase, bachelor, veterinari, medi...",BVMS5 Professional Phase VETMED5042,Veterinary Med and Surgery


In [25]:
model = SentenceTransformer('all-mpnet-base-v2')
temp_sentence = ' '.join(temp_df.iloc[0, 0])
temp_embeding = model.encode(temp_sentence)
print(temp_embeding.shape)

(768,)


In [26]:
embedings = model.encode(temp_df["words"].apply(lambda x: ' '.join(x)))

In [29]:
embedings.shape

(5222, 768)

In [45]:
db = DBSCAN(eps=0.5, min_samples=5).fit(embedings)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
print("Silhouette Coefficient: %0.3f" % silhouette_score(embedings, labels))

Estimated number of clusters: 41
Estimated number of noise points: 2265
Silhouette Coefficient: -0.052


In [50]:
for i in range(1, 10):
  temp_label = np.where(labels == i)
  for j in temp_label:
    print(temp_df.iloc[j][['title', 'dep']])
  print()

                                                title                     dep
50                  Financial Econometrics ACCFIN5217  Accounting and Finance
72    Principles of Financial Econometrics ACCFIN5039  Accounting and Finance
1453  Introductory Statistics for Economists ECON1013               Economics
1543    Research Methods and Research Design ECON5082               Economics
4772           Statistics 3A: Data Analysis STATS3011              Statistics

                                                 title  \
106             Ancient Greek for beginners ADED11389E   
2595                 Greek 1B: Basic Greek 2 GREEK1002   
2596           Greek 2A:Intermediate Greek 1 GREEK2001   
2597          Greek 2B: Intermediate Greek 2 GREEK2002   
2598    Greek 2C: Further Intermediate Greek GREEK2010   
2599                                Greek 2D GREEK2011   
2600                          Advanced Greek GREEK3001   
2616        Intermediate Greek for Honours 1 GREEK4001   
2623     

In [51]:
for i in range(2, 15):
  kmeans = KMeans(n_clusters=i, random_state=0)
  labels = kmeans.fit_predict(embedings)
  print(f"k = {i} Silhouette Coefficient: %0.3f" % silhouette_score(embedings, labels))
i = 68
kmeans = KMeans(n_clusters=i, random_state=0)
labels = kmeans.fit_predict(embedings)
print(f"k = {i} Silhouette Coefficient: %0.3f" % silhouette_score(embedings, labels))

k = 2 Silhouette Coefficient: 0.137
k = 3 Silhouette Coefficient: 0.148
k = 4 Silhouette Coefficient: 0.140
k = 5 Silhouette Coefficient: 0.145
k = 6 Silhouette Coefficient: 0.072
k = 7 Silhouette Coefficient: 0.077
k = 8 Silhouette Coefficient: 0.082
k = 9 Silhouette Coefficient: 0.083
k = 10 Silhouette Coefficient: 0.063
k = 11 Silhouette Coefficient: 0.088
k = 12 Silhouette Coefficient: 0.048
k = 13 Silhouette Coefficient: 0.067
k = 14 Silhouette Coefficient: 0.073
k = 68 Silhouette Coefficient: 0.065


In [53]:
kmeans = KMeans(n_clusters=68, random_state=0)
labels = kmeans.fit_predict(embedings)
for i in range(1, 5):
  temp_label = np.where(labels == i)
  for j in temp_label:
    print(temp_df.iloc[j][['title', 'dep']])
  print()

                                                  title  \
6                 The Accountancy Profession ACCFIN1017   
134                    Drawing and painting 1 ADED11230   
159                Gothic art c.1200 - c.1500 ADED11235   
174          Introduction to Creative Writing ADED11337   
187        Introduction to drawing techniques ADED11239   
...                                                 ...   
4918  A Comprehensive Introduction to Malayalam Gram...   
4954        Chan and Zen Traditions Non Honours TRS3033   
4970                            Body and Belief TRS4105   
4972                    Chan and Zen Traditions TRS4085   
5050  English for Engineering Studies B (UESTC) UEST...   

                                                    dep  
6                                Accounting and Finance  
134                      Adult and Continuing Education  
159                      Adult and Continuing Education  
174                      Adult and Continuing Education  
1

In [95]:
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [84]:
temp_df["label"] = temp_df["dep"].astype("category").cat.codes
x_train, x_test, y_train, y_test = train_test_split(embedings, temp_df["label"], train_size=0.25)

x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(np.array(y_train).reshape(-1, 1))
y_test_scaled = y_scaler.transform(np.array(y_test).reshape(-1, 1))

In [76]:
model = SVC(decision_function_shape='ovo')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.5335716109267297

In [78]:
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.3043145264232831

In [92]:
model = MLPClassifier(
    hidden_layer_sizes=(800, 400, 200, 100, ),
    max_iter=100,
    alpha=1e-4,
    solver="adam",
    verbose=10,
    random_state=1,
    learning_rate_init=0.001,
)
model.fit(x_train_scaled, y_train)

print("Training set score: %f" % model.score(x_train_scaled, y_train))
print("Test set score: %f" % model.score(x_test_scaled, y_test))

Iteration 1, loss = 4.06529268
Iteration 2, loss = 3.86003965
Iteration 3, loss = 3.71108580
Iteration 4, loss = 3.56421361
Iteration 5, loss = 3.41974683
Iteration 6, loss = 3.27235972
Iteration 7, loss = 3.12995764
Iteration 8, loss = 3.01075883
Iteration 9, loss = 2.89437780
Iteration 10, loss = 2.78144588
Iteration 11, loss = 2.67490668
Iteration 12, loss = 2.58004834
Iteration 13, loss = 2.43480129
Iteration 14, loss = 2.31812437
Iteration 15, loss = 2.27552028
Iteration 16, loss = 2.13826853
Iteration 17, loss = 2.06192198
Iteration 18, loss = 2.10431401
Iteration 19, loss = 1.93854894
Iteration 20, loss = 1.91977475
Iteration 21, loss = 1.86560345
Iteration 22, loss = 1.84030729
Iteration 23, loss = 1.70087113
Iteration 24, loss = 1.62012300
Iteration 25, loss = 1.59007511
Iteration 26, loss = 1.54733924
Iteration 27, loss = 1.48281934
Iteration 28, loss = 1.46807415
Iteration 29, loss = 1.43683659
Iteration 30, loss = 1.37514665
Iteration 31, loss = 1.39066621
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Test set score: 0.565484


In [94]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.5128925197855502

In [96]:
model = GaussianNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

0.4738320142966556